#cnn

In [ ]:
import librosa
import numpy as np
from keras.models import Sequential
from keras.layers import Dense, Dropout, Flatten
from keras.layers import Conv1D, MaxPooling1D
from keras.utils import to_categorical
import glob
import os
from sklearn.model_selection import train_test_split

# 주파수 특성 추출 함수 (슬라이딩 윈도우 사용)
max_length = 15  # 각 슬라이딩 윈도우의 길이 (초)
window_hop = 5   # 윈도우를 이동시키는 간격 (초)

def extract_features(audio_files, max_length, window_hop):
    features = []
    for audio_file in audio_files:
        y, sr = librosa.load(audio_file, sr=None)
        sr_per_window = int(sr * max_length)
        sr_hop = int(sr * window_hop)

        # 슬라이딩 윈도우로 자르기
        for i in range(0, len(y) - sr_per_window + 1, sr_hop):
            window = y[i:i+sr_per_window]
            mfccs = librosa.feature.mfcc(y=window, sr=sr, n_mfcc=13)
            chroma = librosa.feature.chroma_stft(y=window, sr=sr)
            spectral_contrast = librosa.feature.spectral_contrast(y=window, sr=sr)
            window_features = np.concatenate((mfccs.flatten(), chroma.flatten(), spectral_contrast.flatten()))
            features.append(window_features)

    # 특성을 배열로 변환
    features = np.array(features)

    return features


def get_files_from_dir(path, file_format):
    # 주어진 경로와 그 하위 경로에서 특정 형식의 파일들을 찾습니다.
    return glob.glob(os.path.join(path, f'**/*.{file_format}'), recursive=True)


ai_subfolders = ['sm', 'sy', 'mh']
ai_audio_files = []
for folder in ai_subfolders:
    ai_audio_files.extend(get_files_from_dir(f'/content/drive/MyDrive/PBL/ai_voice/ai/{folder}', 'mp3'))

# 사람 음성파일들
human_subfolders = ['cy', 'sy', 'hs']
human_audio_files = []
for folder in human_subfolders:
    human_audio_files.extend(get_files_from_dir(f'/content/drive/MyDrive/PBL/ai_voice/human/{folder}', 'mp3'))

ai_features_train_test_fullset = extract_features(ai_audio_files + human_audio_files,
                                                  max_length=max_length,
                                                  window_hop=window_hop)

# 클래스 레이블 생성 및 데이터 분할을 위한 인덱스 계산
num_ai_samples_train_test_fullset=len(ai_audio_files)*((max_length-window_hop)//window_hop+1)
num_human_samples_train_test_fullset=len(human_audio_files)*((max_length-window_hop)//window_hop+1)

y_train_test_fullset=np.concatenate((np.zeros(num_ai_samples_train_test_fullset),
                                     np.ones(num_human_samples_train_test_fullset)))

X_train_test_fullset=np.reshape(ai_features_train_test_fullset,
                                (num_ai_samples_train_test_fullset+num_human_samples_train_test_fullset,-1))

print("Shape of X_train_test:", X_train_test.shape)
print("Shape of y_train_test:", y_train_test.shape)

# 학습 데이터와 테스트 데이터 분리
X_train_test_fullset = np.expand_dims(X_train_test_fullset, axis=2)

# 클래스 수 계산
num_classes = len(set(y_train_test))

# 클래스 레이블 변환
ytrain_ohencd = to_categorical(y_tr, num_classes=num_classes)
yval_ohencd = to_categorical(y_val, num_classes=num_classes)

# 모델 정의 및 컴파일
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=5, strides=1, padding="same", activation="relu", input_shape=(X_train_test_fullset.shape[1], X_train_test_fullset.shape[2])))
model.add(MaxPooling1D(pool_size=5, strides=2, padding='same'))
model.add(Conv1D(filters=32, kernel_size=5, strides=1, padding="same", activation="relu"))
model.add(MaxPooling1D(pool_size=5, strides=2, padding='same'))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(units=num_classes , activation="softmax"))

# 모델 컴파일
model.compile(optimizer='adam', loss='categorical_crossentropy', metrics=['accuracy'])

# 모델 훈련
history = model.fit(X_train_vali,
                    ytrain_ohencd,
                    batch_size=16,
                    epochs=30,
                    validation_data=(X_vali,yval_ohencd))


Shape of X_train_test: (2, 0)
Shape of y_train_test: (0,)


NameError: ignored

In [ ]:
# 모델 정의
model = Sequential()
model.add(Conv1D(filters=64, kernel_size=5, strides=1, padding="same", activation="relu", input_shape=(X_train.shape[1], 1)))
model.add(MaxPooling1D(pool_size=5, strides=2, padding='same'))
model.add(Conv1D(filters=32, kernel_size=5, strides=1, padding="same", activation="relu"))
model.add(MaxPooling1D(pool_size=5, strides=2, padding='same'))
model.add(Dropout(0.2))
model.add(Flatten())
model.add(Dense(units=1, activation="sigmoid"))  # 출력 레이어의 활성화 함수를 sigmoid로 변경

# 이진 분류 레이블 생성
ai_labels = np.zeros(len(ai_features))
human_labels = np.ones(len(human_features))

# 학습 데이터와 테스트 데이터 레이블 결합
y_train = np.concatenate((ai_labels, human_labels))
y_test = np.concatenate((ai_labels, human_labels))

# 모델 컴파일 및 학습 (이진 분류에 대한 설정으로 변경)
model.compile(optimizer='adam', loss='binary_crossentropy', metrics=['accuracy'])

# 데이터 shape 확인
print("Shape of X_train:", X_train.shape)
print("Shape of X_test:", X_test.shape)
print("Shape of y_train:", y_train.shape)
print("Shape of y_test:", y_test.shape)

# 에포크 및 배치 크기 조정
epochs = 10  # 적절한 에포크 수로 변경
batch_size = 32  # 적절한 배치 크기로 변경

history = model.fit(X_train, y_train, batch_size=batch_size, epochs=epochs, validation_data=(X_test, y_test))



NameError: ignored

In [ ]:

# 모델 컴파일 및 학습

X_train = X_train.reshape(X_train.shape[0], X_train.shape[1], 1)
y_train = to_categorical(y_train)

X_test = X_test.reshape(X_test.shape[0], X_test.shape[1], 1)
y_test = to_categorical(y_test)
print("X_train shape:", X_train.shape)
print("X_test shape:", X_test.shape)
print("y_train shape:", y_train.shape)
print("y_test shape:", y_test.shape)

history=model.fit(X_train,y_train,batch_size=16,epochs=30,validation_data=(X_test,y_test))



NameError: ignored

In [ ]:
# 예측 및 결과 출력
test_audio_file = "/content/drive/MyDrive/PBL/ai_voice/ai/vocals_10sec.mp3"
test_features = extract_features(test_audio_file, max_length)
test_features = test_features.reshape(1, test_features.shape[0], 1)
prediction = model.predict(test_features)
predicted_class = np.argmax(prediction)  # 가장 높은 확률을 가진 클래스 선택

confidence = prediction[0][predicted_class] * 100  # 선택된 클래스의 확률

if predicted_class == 0:
    print(f"테스트 음성은 AI 음성입니다. (확신도: {confidence:.2f}%)")
else:
    print(f"테스트 음성은 인간 음성입니다. (확신도: {confidence:.2f}%)")


In [ ]:
import matplotlib.pyplot as plt

# Plot training & validation accuracy values
plt.figure(figsize=(12, 4))

plt.subplot(1, 2, 1)
plt.plot(history.history['accuracy'])
plt.plot(history.history['val_accuracy'])
plt.title('Model accuracy')
plt.ylabel('Accuracy')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')

# Plot training & validation loss values
plt.subplot(1, 2, 2)
plt.plot(history.history['loss'])
plt.plot(history.history['val_loss'])
plt.title('Model loss')
plt.ylabel('Loss')
plt.xlabel('Epoch')
plt.legend(['Train', 'Test'], loc='upper left')

# Show the plot
plt.tight_layout()
